# Python Tutorial: Quickly Make Beautiful Interactive Maps - Zoom Out

This is the third part in our four part series on how to use python to quickly create interactive maps.   

In Part 1 we demonstrated how to use Python to create a `Map` object, locate it, decide the zoom range, tiles and add a layer control feature.  
In part 2 we showed how to create `Marker`s and `Circle`s objects, group them into a layer and customise their image and text. 

Here we focus on layer features used most commonly in a zoomed-out map:

* Heatmaps: static or changing with time.
* Choropleths  

(The API used here assumes version `0.5.0`.)

We conclude with a discussion on which feature is best for different use cases (i.e, choosing between markers, circles, heatmaps, choropleths).



**Disclaimer**  
He we discuss only a handful of features which we use frequently. The reader is encouraged to use the module help pages to explore the full potential and updates in the API.  
(The Folium API used here assumes version `0.5.0`.)  

Maps used are provided by © OpenStreetMap contributors. 

In [1]:
import folium  # to install: pip install folium

folium.__version__

u'0.5.0'

## Data  

To set the stage we will introduce some data that we want to put on a heatmap.   
(All points are located in New York City)   

In [22]:
import numpy as np

center_latitude, center_longitude = 40.729183, -73.994263
ngeos = 1000

jitterDelta = 0.00022
jitterMin, jitterMax = 1. - jitterDelta, 1. + jitterDelta

np.random.seed(1)
latitudes = np.random.uniform( (center_latitude * jitterMin), (center_latitude * jitterMax), ngeos)
longitudes = np.random.uniform( (center_longitude * jitterMin), (center_longitude * jitterMax), ngeos)

latitude_longitudes = zip(latitudes, longitudes)

print "`latitude_longitudes` contains {:,} geo points,\nwhere the first 5 entries are:".format(len(latitude_longitudes))  
latitude_longitudes[0:5]

`latitude_longitudes` contains 1,000 geo points,
where the first 5 entries are:


[(40.727695964579603, -73.988591812217351),
 (40.733131400109627, -74.006954794204177),
 (40.720224629432856, -74.002457968012365),
 (40.725640633558136, -74.002813638070748),
 (40.722852568654702, -73.993269314213393)]

## HeatMap (regular)  

We will create a heatmap layer using `folium.plugins.HeatMap`.  

Useful keywords:  
* `radius` - radius of each geopoint (in pixels)
* `blur` - how much to blur the points for a visual effect
* `gradient` - Color gradient for blobs (default `None`) a dictionary of values. E,g `{0.4: 'blue', 0.65: 'lime', 1.: 'red'}`

In [17]:
from folium import plugins

In [18]:
# ====== Create a Map object, location and zoom ======
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
zoom_start = 16

map_ = folium.Map(location=location, zoom_start=zoom_start, control_scale=True) 

# ====== Create the HeatMap layer ======
radius_pixels = 20 
blur = 10 #20
gradient = None 
heatmap = plugins.HeatMap(latitude_longitudes, name = "Heatmap", radius=radius_pixels, blur=blur, gradient=gradient)

# ====== Add Heatmap Layer to the map ======
heatmap.add_to(map_) # or map_.add_child(heatmap)

# ====== Display ======
# map_.save('my_map.html') # (alternatively create HTML)

map_ # display in notebook

Notes:  
* Heatmaps represent ***relative intensity*** of the metric (in our case geo points) with the objective to attract the eye to a region.    
* Play with `radius` (e.g set to 10) and `blur` (e.g set to 0) to see what happens  

## HeatMap (with time)  

Another cool option is a heatmap as a function of time using `folium.plugins.HeatMapWithTime`.  
This is useful to show how data evolves as a function of time, e.g, to follow a trend or an epidemic. 

** Data Preparation **

For the purposes of the current API you need to create a list of lists of locations, where the locations have to be a list of longitude-latitude (not a tuple like before).   

In [26]:
# Example data
l_points = map(lambda x: list(x),  latitude_longitudes) # must be a list of list of lists

# steps names (needs to be same length as l_l_points)
index_steps  = ["Monday", 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

l_l_points = [] # needs to be the same size as index_steps
for i in range(len(index_steps)):
    l_l_points.append( l_points[:(20 * i + 1)] + l_points[-(20 *i + 2):-1])

print "Note:\nThe input data should be a list of lists of lists\n"
print "In our example\n`l_l_points` contains {} step lists of geo coordinates, \nwhere each geo location contains {} coordinates".format(len(l_l_points), len(l_l_points[-1][0]))
print "The {} steps are labeled as: {}".format(len(l_l_points), index_steps)
print '-' * 20
print "E.g, the step labelled '{}' contains {:,} geo locations that are in a list of lists (showing first 5 entries):".format(index_steps[-1], len(l_l_points[-1]))
l_l_points[-1][:5]

Note:
The input data should be a list of lists of lists

In our example
`l_l_points` contains 7 step lists of geo coordinates, 
where each geo location contains 2 coordinates
The 7 steps are labeled as: ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
--------------------
E.g, the step labelled 'Sunday' contains 242 geo locations that are in a list of lists (showing first 5 entries):


[[40.727695964579603, -73.988591812217351],
 [40.733131400109627, -74.006954794204177],
 [40.720224629432856, -74.002457968012365],
 [40.725640633558136, -74.002813638070748],
 [40.722852568654702, -73.993269314213393]]

** Map Creation ** 

In [30]:
radius_pixels = 100
auto_play = True # True means that the animation will start automatcally
heatmap_wTime = plugins.HeatMapWithTime(l_l_points, index_steps, radius=30, auto_play=auto_play, display_index=True)

# ====== Create a map object, location and zoom ======
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
zoom_start = 15

map_ = folium.Map(location=location, zoom_start=zoom_start) 


# ====== Add Marker Group Layer to the map ======
heatmap_wTime.add_to(map_)

# ====== Display ======
map_.save('my_map.html') # (alternatively create HTML)

map_ # display in notebook

Notes: 
* Set `auto_play=True` to have the animation start automatically.    
* The user has a loop button (the  `v0.5.0` API does not have a toggle option).  


## Choropleths

Choropleths display metrics of a theme by region, e.g ZIP Code income per capita, County unemployement rate.  

Here we show how to create a choropleth for a given shapefile or geoJSON.  

This section is divided into two parts 

* Reading and manipulating shape file data to a geoJSON  
* Creating a `folium.GeoJson` object as a choropleth layer in a map.    

### Shape Data

For the purpose of this example we use the US 2010 Census New York state ZIP Code shapefiles `tl_2010_36_zcta510` in the US Census data (see below download instructions).  
(It's actually Census ZCTA which is similar to US Post ZIP Code, see [Wikipedia](https://en.wikipedia.org/wiki/ZIP_Code_Tabulation_Area) for details.)  

Here we:  
* Open and read the shape file with  `shapefile` from [pyshp](https://pypi.python.org/pypi/pyshp) 
* Create a geoJSON file with  `json.dumps`  
* Read the geoJSON file with [geopandas](http://geopandas.org/).  
This yields a `geopandas.GeoDataFrame` object which is simlar in usage to the `pandas.DataFrame`  
* Sample a handful of ZIP Codes to explore and examine them on a map

**Data Download Instructions**  
Go to [www.census.gov/cgi-bin/geo/shapefiles2010/main](https://www.census.gov/cgi-bin/geo/shapefiles2010/main) and do the following:  
* Under "Select Layer Type" choose "ZIP Code Tabulation Area" and submit.   
* Under "5-Digit ZIP Code Tabulation Area (2010) " select "New York" and download and place the `tl_2010_36_zcta510` folder in the `path` directory of your choice.  

### ShapeFile to GeoJson

In [31]:
import shapefile # pip install pyshp (for opening the shape file)
from json import dumps # (for creating a geoJSON file)
import geopandas # pip install geopandas (for reading a geoJSON file)

In [32]:
# Defining file paths 

path = "../data/"
shape_path = "{}/tl_2010_36_zcta510/tl_2010_36_zcta510.shp".format(path)
geojson_path = "{}/tl_2010_36_zcta510.geojson".format(path)

In [111]:
# Opening the Shape File

reader = shapefile.Reader(shape_path)
fields = reader.fields[1:]
field_names = [field[0] for field in fields]

buffer_ = []
for sr in reader.shapeRecords():
       atr = dict(zip(field_names, sr.record))
       geom = sr.shape.__geo_interface__
       buffer_.append(dict(type="Feature", geometry=geom, properties=atr)) 

In [112]:
# Creating a geoJSON file 

geojson = open(geojson_path, "w")
geojson.write(dumps({"type": "FeatureCollection", "features": buffer_}, indent=2) + "\n")
geojson.close()

In [33]:
# Reading in the geoJSON to a GeoDataFrame

gdf_geo = geopandas.read_file(geojson_path)
gdf_geo.set_index('ZCTA5CE10', inplace=True) # ZIP Codes (ZCTA5CE10) are unique, hence setting as index

print gdf_geo.shape
gdf_geo.head(6)

(1794, 11)


AWATER10 PARTFLG10 CLASSFP10    ALAND10 STATEFP10 FUNCSTAT10  \
ZCTA5CE10                                                                 
12205        243508         N        B5   40906445        36          S   
12009       2168637         N        B5  135241924        36          S   
14804        232123         N        B5  144718714        36          S   
14836        131305         N        B5   77612958        36          S   
14536        425175         N        B5   47193482        36          S   
10464        236605         N        B5    9070627        36          S   

            INTPTLAT10 MTFCC10  GEOID10    INTPTLON10  \
ZCTA5CE10                                               
12205      +42.7187855   G6350  3612205  -073.8292399   
12009      +42.6975663   G6350  3612009  -074.0355422   
14804      +42.3172588   G6350  3614804  -077.8479358   
14836      +42.5429182   G6350  3614836  -077.8781933   
14536      +42.5439751   G6350  3614536  -078.0836709   
10464      +40.8677868   G6350  3610464  -073.7999204   

                                                    geometry  
ZCTA5CE10                                                     
12205      POLYGON ((-73.87051699999999 42.751227, -73.86...  
12009      POLYGON ((-74.10891099999999 42.653004, -74.10...  
14804      POLYGON ((-77.92747 42.34775399999999, -77.926...  
14836      (POLYGON ((-77.95599299999999 42.474325, -77.9...  
14536      POLYGON ((-78.050297 42.538502, -78.0502429999...  
10464      (POLYGON ((-73.78619399999999 40.873886, -73.7...

### Examining the Shapes
Before creating the Choropleth, let's make sure that the polygons display nicely on the map.  
Here we introduce our first usage of the `folium.GeoJson` object. 

In [34]:
# ====== GeoDataFrame sample ======
l_zips = ['10002', '10003', '10009','10011', '10012', '10013','10014', '11211', '11222']
gdf_sample = gdf_geo.loc[l_zips]

# ====== initiating map ======
zoom_start = 14
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
map_ = folium.Map(location=location, zoom_start=zoom_start, control_scale=True)

# ====== Creating GeoJson object ======
geojson_ = folium.GeoJson(gdf_sample) # example GeoJson used

# ====== Adding GeoJson layer to map ======
geojson_.add_to(map_) # or: map_.add_child(geojson_)

# ====== Display ======
# map_.save('my_map.html') # (alternatively create HTML)
map_

### Creating a Choropleth in a few steps

Our main objective is to link eash shape ID to a color.    
In our case we are mapping the index of our `GeoDataFrame` (`gdf_sample.index`) to a color.  
This is done via the `style_function` keyword.  

The steps are:  

* Create a dictionary mapping from shape ID to value to color (e.g, `dict_id2color`)      
* Define `style_function` - 
    * maps between IDs and colors (keyword `fillColor`).  
    * Styles features such as `opacity`, line `color`.    
* Create a `folium.GeoJson` object and layer add to a map 
* (Optional) Set a `highlight_function` - this enable highlighting a region when hovered over with the mouse.  

In [35]:
# Step 1: Create a mapping between the ID of a shape to a color
dict_id2color =    { '10002': 'green',
                     '10003': 'green',
                     '10009': 'yellow',
                     '10011': 'red',
                     '10012': 'yellow',
                     '10013': 'green',
                     '10014': 'red',
                     '11211': 'red',
                     '11222': 'yellow'}


# Step 2: Create a style_function  
def my_style_function(feature):
    return {
        'fillColor': dict_id2color[feature['id']],
        'color': 'black', # line color
        'weight': 2,      # line width
        'opacity': 0.7,   # line opacity 
        'fillOpacity': 0.7  # fill color opacity
    }    

The main aspect in style_function (in our example `my_style_function`) is obtaining the color for each `feature['id']`, where `feature` is a shape.  

Now that we set up the style_function let's see what it looks like on a map.  

In [36]:
# ====== initiating map ======
zoom_start = 14
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
map_ = folium.Map(location=location, zoom_start=zoom_start)


# ==== Creating GeoJson object with style_function ======
geojson_ = folium.GeoJson(gdf_sample,
                          style_function= my_style_function
                         ) 
# ==== Add GeoJson to map as layer ======
map_.add_child(geojson_) # or: geojson_.add_to(map_)

#map_ = map_.add_child(colors_linear) # and the color bar

# ====== Display ======
# map_.save('my_map.html') # (alternatively create HTML)

map_

This is a good first step, but you might also want to add to more options:  
* Mouse-over-shape highlighting      
* Legend  

** Highlighting **  
For the Mouse-over-shape highlight, all we need to to create a highlight_function. 

In [37]:
# ======= highlight_function ============ 
def my_highlight_function(feature):
    return {
        'fillColor': dict_id2color[feature['id']],
        'color': 'white',
        'weight': 3,
        'dashArray': '0, 0',
        'opacity': 1.,
        'fillOpacity': 1. 
    }


# ====== initiating map ======
zoom_start = 14
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
map_ = folium.Map(location=location, zoom_start=zoom_start)


# ==== Creating GeoJson object with style_function ======
geojson_ = folium.GeoJson(gdf_sample, 
                          style_function= my_style_function, 
                          highlight_function=my_highlight_function
                         ) 
# ==== Add GeoJson to map as layer ======
map_.add_child(geojson_) # or: geojson_.add_to(map_)

#map_ = map_.add_child(colors_linear) # and the color bar

# ====== Display ======
# map_.save('my_map.html') # (alternatively create HTML)

map_

Hover the mouse above an area and see it highlight by your customization. If you click on the area the map should zoom in to make the area front and center.

** Legend **  
There is no obvious API feature to add a legend, so here we'll learn how to create a `folium.Element` to add objects with minimal JavaScript.   

In [109]:
# This is all it takes to create a small legend 

legend_html = """ 
    <div style=
    "position: fixed;background-color: rgba(255, 255, 255, 0.5); border-radius: 5px;
     bottom: 50px; left: 10px; width: 120px; height: 120px; border:2px solid grey; z-index:9999; font-size:14px;
     ">
     <p style="text-align:center;"> <b> Most Popular </b> </p>
     <p style="margin-left:5px"> <i class="fa fa-square fa-1x" style="margin-right:5px;color:green;"></i> Oscar </p>
     <p style="margin-left:5px"> <i class="fa fa-square fa-1x" style="margin-right:5px;color:yellow;"></i> Big Bird </p>
     <p style="margin-left:5px"> <i class="fa fa-square fa-1x" style="margin-right:5px; color:red;"></i> Elmo </p>
  </div>
     """

legend_layer = folium.Element(legend_html)

**JaveScript tips**  
Most of the above should be clear (by context or trial-and-error), but just to emphasize the not so obvious:  
`<br>` - break of line (like `\n`)  
`&nbsp;` - means one white space  
`z-index:9999` - each layer has a `z-index`, starting from 0. Here we verify that the layer of the legend is high.  
`<b> ... </b>` - is boldface  
`<p> ... </p>` - is a paragraph  
`<i> ... </i>` - adding an image, our can Fontawsome's   `fa-square`.  



The new line in our Folium API procedure below is  

`map_.get_root().html.add_child(legend_layer)`

In [110]:
# ====== initiating map ======
zoom_start = 14
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
map_ = folium.Map(location=location, zoom_start=zoom_start, control_scale=True)


# ==== Creating GeoJson object with style_function ======
geojson_ = folium.GeoJson(gdf_sample, 
                          style_function= my_style_function, 
                          highlight_function=my_highlight_function
                         ) 
# ==== Add GeoJson to map as layer ======
map_.add_child(geojson_) # Note that this is the same as geojson_.add_to(map_)

# Adding Legend
map_.get_root().html.add_child(legend_layer)

# ====== Display ======
# map_.save('my_map.html') # (alternatively create HTML)

map_

<img src="http://weclipart.com/gimg/7F317FA8E8587561/il_fullxfull.276944863.jpg" width="200" height="200" />



** Trouble Shooting **  
If the legend does not appear, you will have to check your Javascript for an incorrect character.

** Continuous Metric **

Above we showed how to create a choropleth to show categorical results.  
In our example we showed which Sesame Street character is most likely to win a popularity vote in each ZIP Code.  


If, however, we would like to focus on the ballot results of one character, e.g, Elmo, you might want to plot a linear color scaling.  

Here we show how this is done in a few steps:  
* Define the mapping between the metric to a color range that will be interpolated with `branca.colormap.LinearColormap` (see our `colors_linear`).  
* Map the IDs to the color (via the metric) (see our `dict_id2color` below)  
* Use the `style_function` as before.  
* Create a color bar legend with `colors_linear.add_to(map_)` or `map_.add_child(colors_linear)`.  




In [45]:
import branca.colormap as cm # branca should be installed with folium

l_colors = [  
    'white', # adding white for contrast
    '#ffffb2',# yellow to red
    '#fed976',
    '#feb24c',
    '#fd8d3c',
    '#fc4e2a',
    '#e31a1c',
    '#b10026']

# choose minimum and maximum values they represent
vmin = 0
vmax = 60.

colors_linear = cm.LinearColormap(l_colors,  vmin=vmin, vmax=vmax)

colors_linear.caption = "Elmo\\'s popular vote percentage"

colors_linear

In [50]:
# ==== Example metric data (ID->metric) ====
import numpy as np

np.random.seed(1)
dict_id2metric = {gdf_sample.index[i]: value for i, value in enumerate(np.random.uniform(vmin, vmax, len(gdf_sample)))}

print "ID -> metric"
print dict_id2metric

# ==== Example color data (ID->color) ====
dict_id2color = {k:colors_linear(v) for k, v in dict_id2metric.iteritems()} 

print '-' * 20
print "ID -> color"
dict_id2color

ID -> metric
{'10002': 25.021320282154441, '10003': 43.219469606529486, '10009': 0.0068624890406931982, '11211': 20.733643622582864, '10013': 5.540315686127868, '10012': 8.8053534490267822, '10011': 18.139954357910387, '11222': 23.806048453840198, '10014': 11.175612682660255}
--------------------
ID -> color


{'10002': '#feb54f',
 '10003': '#fb4c29',
 '10009': '#ffffff',
 '10011': '#fed571',
 '10012': '#fffeb1',
 '10013': '#ffffce',
 '10014': '#fff4a0',
 '11211': '#fec964',
 '11222': '#febb55'}

Now that we have all our ducks in a row:  
* `my_style_function` - contains `dict_id2color` to map the IDs to colors  
* `my_highlight_function` - (optional) add mouse-hover-highlight feature  
* `colors_linear` - we used to map between the metrics and colors; here we also use to create a colorbar.  

Let's see how popular Elmo is (according to our random number generator)  

<img src="https://cdn.drawception.com/images/panels/2012/4-2/G4ekYbyNTj-2.png" width="200" height="200" />


In [57]:
# ====== initiating map ======
zoom_start = 14
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
map_ = folium.Map(location=location, zoom_start=zoom_start)


# ==== Creating GeoJson object with style_function ======
geojson_linear = folium.GeoJson(gdf_sample,
                                style_function= my_style_function, 
                                highlight_function=my_highlight_function,
                                name='Elmo\'s popularity Choropleth'
                               ) 
# ==== Add GeoJson to map as layer ======
map_.add_child(geojson_linear) # Note that this is the same as geojson_.add_to(map_)

# ==== Colorbar as a Legend ====
colors_linear.add_to(map_) # or: map_.add_child(colors_linear)

# ====== Display ======
# map_.save('my_map.html') # (alternatively create HTML)

map_

More on color coding is described in [this tutorial](https://github.com/python-visualization/folium/blob/master/examples/Colormaps.ipynb).    


## Heatmap and Choropleth Summary

In this third part of our four part post we explored adding to a `Map` object a regular `HeatMap` or one varying with time (`HeatMapWithTime`) and `GeoJson` layers to create choropleths, all which are useful mostly when zooming out of a map.  

As discussed in part 2, adding geo-points to a heatmap causes it to grow linearly. If dealing with many points, one suggestion to resolve the size issue is to sample. We calculate the cost of each geo-point to be 40 Bytes, which suggests that a 1Mbyte map may contain 25,000 geo points.    

If the fourth and final part of this post we present a cheatsheat, or a reference page.  


# The Kichen Sink Map  

The following summarises most of the features presented in this three part post.  

Here we display layers on a `Map` object:  
* `FeatureGroup` layer of `Marker`s  
* `MarkerCluster` layer of `Circle`s   
* `HeatMap`    
* chorpoleth from `GeoJson` with mouse-hovering-highlight feature  
* `LayerControl`  
* Color bar   
* Custom layer with your own Javascript embeded via a `Element` object.  


In [128]:
# ====== Create a FeatureGroup Layer ======
# Here we group folium.Marker with folium.FeatureGroup, but could be replaced with plugins.MarkerCluster 

marker_group = folium.FeatureGroup(name='Marker FeatureGroup') 

n_markers = 20

l_icons = ['shopping-cart', 'bed', 'building' ] # from fa (fontawesome: https://fontawesome.com/icons)

for idx, latitude_longitude in enumerate(latitude_longitudes[:n_markers]):
    # ===== updating for each icon =======
    icon =  l_icons[idx % len(l_icons)]
    color = 'black' # color of marker
    icon_color = 'white' # color of marker drawing
    prefix = 'fa' #'glyphicon' #'fa'
    
    icon = folium.Icon(icon=icon, prefix=prefix, color=color, icon_color=icon_color)
    # ==================================== 
    
    label = "Marker No. {}".format(idx) # we can label each label with popup 
    folium.Marker(latitude_longitude, popup=label, icon=icon).add_to(marker_group)

In [129]:
from folium import plugins
# Here we group folium.Circle with plugins.MarkerCluster, but could be replaced with folium.FeatureGroup

marker_group_clustered = plugins.MarkerCluster(name='Circle \'MarkerClusters\'')

n_markers = 20

l_icons = ['shopping-cart', 'bed', 'building' ] # from fa (fontawesome: https://fontawesome.com/icons)

for idx, latitude_longitude in enumerate(latitude_longitudes[n_markers:n_markers*4]):
    label = "<b>Circle</b><br>No. {}".format(idx) # we can label each label with popup 
    folium.Circle(latitude_longitude, popup=label, fill=True, fill_opacity=1., radius=10, color='green').add_to(marker_group_clustered)

In [133]:
# Custom Layer, in our case a legend

legend_html_kitchen = """ 
    <div style=
    "position: fixed;background-color: rgba(255, 255, 255, 0.5); border-radius: 5px;
     bottom: 50px; left: 10px; width: 120px; height: 120px; border:2px solid grey; z-index:9999; font-size:14px;
     ">
     <p style="text-align:center;"> <b> Legend </b> </p>
     <p style="margin-left:5px"> <i class="fa fa-shopping-cart fa-1x" style="margin-right:5px;color:black;"></i> Supermarket </p>
     <p style="margin-left:5px"> <i class="fa fa-bed fa-1x" style="margin-right:5px;color:black;"></i> Hotel </p>
     <p style="margin-left:5px"> <i class="fa fa-building fa-1x" style="margin-right:5px; color:black;"></i> Office </p>
  </div>
     """

legend_layer_kitchen = folium.Element(legend_html_kitchen)

In [134]:
heatmap = plugins.HeatMap(latitude_longitudes[n_markers*2:n_markers*10], name = "Heatmap", 
                          radius=20, blur=blur, 
                          gradient=None)

In [138]:
# ===== Minimum required to use your own tile =====
map_ = folium.Map(location=location, zoom_start=14, tiles=None, control_scale=True)
map_.add_tile_layer(tiles='Stamen Terrain' , name='Stamen Terrain tiles'  )

# ===== Add this if you want more tile option for the user ======
map_.add_tile_layer(tiles='CartoDB positron', name='CartoDB positron tiles')

# ==== Add GeoJson to map as layer ======
map_.add_child(geojson_linear) # Note that this is the same as geojson_.add_to(map_)

# ==== Colorbar as a Legend ====
colors_linear.add_to(map_) # or: map_.add_child(colors_linear)

# ====== Adding Marker FeatureGroup Layer ======
map_.add_child(marker_group) # or: marker_group.add_to(map_)

# ====== Adding MarkerCluster Layer ======
map_.add_child(marker_group_clustered)

# ====== Add Heatmap Layer ======
heatmap.add_to(map_) # or map_.add_child(heatmap)

# ===== Layer Control ======
layer_control = folium.LayerControl(position='topleft', collapsed=False)
map_.add_child(layer_control) # adding Layer Control (optional). Alternatively: layer_control.add_to.(map_)

# Adding Legend
map_.get_root().html.add_child(legend_layer_kitchen)

map_

Here we see that the API shows all the layers. Unforutnately it does not let you toggle in advance which ones to show. As of version `0.50`, this would require a Javascript hack. 